In [ ]:
!pip install tensorflow==2.12.0
!pip install transformers==4.6.0
!pip install protobuf==3.20.*

In [ ]:
pip install yfinance scikit-learn pandas numpy transformers tensorflow speechrecognition nltk langdetect pyaudio sentencepiece torch streamlit streamlit-webrtc gtts spacy langchain openai playsound

Citations:<br>
The Power of Generative AI: A Review of Requirements, Models, Input–Output Formats, Evaluation Metrics, and Challenges by Ajay Bandi, Pydi Venkata Satya Ramesh Adapa and Yudu Eswar Vinay Pratap Kumar Kuchi<br>
 
2106.12985v2 Stock Market Analysis with Text Data: A Review Kamaladdin Fataliyev, Aneesh Chivukula, Mukesh Prasad, Wei Liu<br>
 
Stock market prediction using artificial intelligence: A systematic review of systematic reviews Chin Yang Lin, João Alexandre Lobo Marques<br>
 
A Study Analyzing an Innovative Approach to Sentiment Analysis with VADER by Chadha, Raman and Chaudhary, Aryan, Journal of Engineering Design and Analysis<br>

Medium : A Beginner’s Guide to using AI in Stock Prediction with Google Colab and Tableau
https://medium.com/@larry.deee/a-beginners-guide-to-using-ai-in-stock-prediction-with-google-colab-and-tableau-5e63153d3371

Medium : Using Sentiment Analysis to Predict the Stock Market
https://medium.com/analytics-vidhya/using-sentiment-analysis-to-predict-the-stock-market-77100295d753#:~:text=Using%20Sentiment%20Analysis%20to%20Predict%20the%20Stock%20Market,score%20index%20against%20the%20daily%20close%20return%20

Medium : Ultimate Guide to Getting Started with LangChain
https://medium.com/@zilliz_learn/ultimate-guide-to-getting-started-with-langchain-b9a87cb340f8

Stock Related functions

Implementation Steps<br>
Step 1: Firstly, we must install the necessary packages via pip and configure the required api keys<br>
Step 2: Then load the required models from spacy and Vader to handle the NLP tasks and sentiment analysis tasks. <br>
Step 3: Define Functions to load the ticker data from excel and the stock data from api <br>
step 4: Setup Langchain and integrate with the transformer model and suitable prompt template to generate the summary <br>
Step 4: We define a function for speech recognition to get the input from the user and convert it to text. <br>
Step 5: Simultaneously fetch the entities from the voice input and hit the apis to get sufficient amount of data related to news articles as well as financial data to be ready for further text processing.<br>
Step 6: Once we receive data from the api, we will do the sentimental analysis and generate a compound sentimental value.<br>
Step 7: We do input the historical data of the stock and the sentimental analysis from Vader to the langchain where it will be having suitable prompt templates and language model to summarize about the recommendation.


Numpy: It is a fundamental package for scientific computing with Python. It provides support for arrays, matrices, and a large collection of mathematical functions to operate on these data structures.<br>
Spacy: An open-source library for advanced Natural Language Processing (NLP) in Python. It’s designed specifically for production use and helps in extracting entities and understanding text.<br>
Vader: A lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.<br>
News API: A simple HTTP REST API for searching and retrieving live articles from all over the web. It can be used to get news data based on keywords, sources, and other parameters.<br>
Alpha Vantage API: Provides real-time and historical stock market data. This API can be used for financial analysis, market research, and more.

In [48]:
import yfinance as yf
import pandas as pd
import numpy as np
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import requests
from langdetect import detect, LangDetectException
import spacy
nlp = spacy.load("en_core_web_sm")
import nltk
import json
nltk.download('vader_lexicon')

#     "amazon": "AMZN",
#     "apple": "AAPL",
#     "google": "GOOGL",
#     "microsoft": "MSFT",
#     "facebook": "META",
#     "tesla": "TSLA",
#     "netflix": "NFLX",
#     "paypal": "PYPL",
#     "visa": "V",
#     "salesforce": "CRM"
FINANCIAL_NEWS_SOURCES = [
    "bloomberg.com",
    "cnbc.com",
    "financialpost.com",
    "financialexpress.com",
    "marketwatch.com",
    "reuters.com",
    "wsj.com",
    "yahoo.com",
]

def fetch_historical_data(ticker, period='1mo'):
    stock_data = yf.download(ticker, period=period)
    return stock_data

def fetch_realtime_data(ticker):
    stock_data = yf.download(ticker, period='1d', interval='1m')
    return stock_data

def extract_companyName(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "ORG":
            return ent.text
    return None

def extract_ticker(companyName):
        return company_tickers.get(companyName.lower(), None)

def load_company_tickers(csv_file):
    df = pd.read_csv(csv_file)
    company_ticker_dict = pd.Series(df.ticker.values, index=df.company).to_dict()
    return company_ticker_dict

def extract_tickerFromExcel(companyName):
    company_tickers = load_company_tickers('company-ticker.csv')
    return company_tickers.get(companyName.lower(), None)

# def predict_stock_price(history):
#     predictor = pipeline('text-generation', model='distilgpt2')
#     history_str = ' '.join([f"{date}: {price}" for date, price in history.items()])
    
#     prompt = f"Predict the closing price for the next trading day based on the following history: {history_str}."
#     max_input_length = predictor.model.config.n_positions - 20
#     if len(prompt) > max_input_length:
#         prompt = prompt[:max_input_length]
#     prediction = predictor(prompt, max_new_tokens=20, num_return_sequences=1)
#     predicted_price = float(re.findall(r"\d+\.\d+", prediction[0]['generated_text'])[0])
#     return predicted_price

def fetch_news(company_name):
    try:
     with open('config.json') as config_file:
      config = json.load(config_file)
      api_key = config['newsAPI']
    except:
        print("Unable to access stock Data")
    query = f"{company_name}"
    domains = ','.join(FINANCIAL_NEWS_SOURCES)
    url = f"https://newsapi.org/v2/everything?q={query}&domains={domains}&sortBy=publishedAt&apiKey={api_key}"
    response = requests.get(url)
    print(url)
    news_data = response.json()
    if news_data["status"] == "ok":
        articles = news_data["articles"]
        return articles
    else:
        print(f"Failed to fetch news: {news_data['message']}")
        return []


def get_ticker_symbol_yahoo(common_name):
    base_url = f"https://query1.finance.yahoo.com/v1/finance/search?q={common_name}"
    response = requests.get(base_url)
    
    if response.status_code != 200:
        return f"Failed to fetch data: {response.status_code}"

    try:
        data = response.json()
    except ValueError:
        return "Failed to parse JSON response"

    if 'quotes' in data and data['quotes']:
        return data['quotes'][0]['symbol']
    else:
        return "Ticker symbol not found"
    
def get_ticker_symbol_yfinance(common_name):
    try:
        tickers = yf.Ticker(common_name)
        info = tickers.info
        
        if 'symbol' in info:
            return info['symbol']
        else:
            return "Ticker symbol not found"
    except Exception as e:
        return f"An error occurred: {e}"
    
def get_ticker_symbol(common_name):
    api_key = ""
    base_url = "https://finnhub.io/api/v1/search"
    params = {
        'q': common_name,
        'token': api_key
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    if data['count'] > 0:
        for result in data['result']:
            if common_name.lower() in result['description'].lower() and result['type'] == 'Common Stock':
                print(result)
                return result['symbol']
        return "Ticker symbol not found"
    else:
        return "Ticker symbol not found"

def fetch_daily_stock_data(symbol):
    try:
     with open('config.json') as config_file:
      config = json.load(config_file)
      api_key = config['alphavantage']
    except:
        print("Unable to access stock data")


    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}&outputsize=compact'
    response = requests.get(url)
    data = response.json()
    
    if "Time Series (Daily)" in data:
        time_series = data["Time Series (Daily)"]
        stockData = pd.DataFrame.from_dict(time_series, orient='index')
        stockData = stockData.rename(columns={
            "1. open": "Open",
            "2. high": "High",
            "3. low": "Low",
            "4. close": "Close",
            "5. volume": "Volume"
        })
        stockData.index = pd.to_datetime(stockData.index)
        stockData = stockData.sort_index()

        return stockData
    else:
        print("Error fetching data:", data)
        return None
    
def analyze_sentiment(articles):
    try:
        sia = SentimentIntensityAnalyzer()
    except LookupError:
        nltk.download('vader_lexicon')
        sia = SentimentIntensityAnalyzer()
        
    sentiments = []
    for article in articles:
        title = article.get('title') or ''
        description = article.get('description') or '' 
        if title or description: 
            combined_text = title + ' ' + description
            try:
                if detect(combined_text) == 'en': 
                    sentiments.append(sia.polarity_scores(combined_text))
            except LangDetectException:
                continue  
    if not sentiments: 
        return {'neg': 0, 'neu': 1, 'pos': 0, 'compound': 0}
    
    avg_sentiment = {
        'negative': np.mean([s['neg'] for s in sentiments]),
        'neutral': np.mean([s['neu'] for s in sentiments]),
        'positive': np.mean([s['pos'] for s in sentiments]),
        'average': np.mean([s['compound'] for s in sentiments])
    }
    return avg_sentiment

def predict_stock_movement(sentiment):
    if sentiment['average'] > 0.05:
        return "Invest"
    elif sentiment['average'] < 0:
        return "Do not invest"
    else:
        return "Hold"
    
def decide_investment(predicted_price, current_price):
    if predicted_price > current_price:
        return "Invest"
    else:
        return "Do not invest"

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\s567075\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Transformer model related functions<br>
Transformers: Provided by Hugging Face, this library is used for state-of-the-art NLP. It allows you to use pre-trained models for tasks like text summarization, translation, and more.<br>
LangChain: This framework helps manage interactions with large language models, handling prompt templates and chaining multiple model calls.<br>

In [43]:
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from transformers import pipeline, TextGenerationPipeline
import os
import json
generator = pipeline("text-generation", model="gpt2")
class CustomTextGenerationPipeline(TextGenerationPipeline):
    def __call__(self, *args, **kwargs):
        kwargs['max_new_tokens'] = kwargs.get('max_new_tokens', 100)
        kwargs['temperature'] = kwargs.get('temperature', 0.5)
        return super().__call__(*args, **kwargs)

custom_generator = CustomTextGenerationPipeline(
    model=generator.model,
    tokenizer=generator.tokenizer,
    framework=generator.framework,
    task=generator.task,
    device=generator.device
)
llm = HuggingFacePipeline(pipeline=custom_generator) 



def generate_summary(sentiment, decision,companyName,history):
 sentiment_text=""
 if decision == "Invest":
        if sentiment["average"] > 0.5:
            sentiment_text = f"The overall sentiment is positive, indicating a potentially favorable investment climate."
        else:
            sentiment_text = f"While the sentiment leans slightly positive, further analysis is recommended to confirm the investment's viability."
 else:
     sentiment_text = f"Sentiment analysis indicates to be cautious about {companyName}"
 try:
  with open('config.json') as config_file:
    config = json.load(config_file)
    api_keyOpenAI = config['openAI']
 except:
  print("Unable to access stock data")
 llm = OpenAI(temperature=0.7, api_key=api_keyOpenAI) 


 prompt_template = PromptTemplate(
    input_variables=["negative", "neutral", "positive", "average", "decision","companyName","history","sentimenttext"],
    template=(
        "The sentiment analysis of recent news articles about the company reveals:\n"
        "  * Negative sentiment: {negative:.2f}\n"
        "  * Neutral sentiment: {neutral:.2f}\n"
        "  * Positive sentiment: {positive:.2f}\n"
        "  * compound sentiment: {average:.2f}\n"
        "Based on this analysis, the investment decision is: {decision}.\n The compound sentiment of {companyName} is {average:2f} I analysed that {sentimenttext} predict its stock performance based on the history provided ({history}) and above provided sentiment analysis"
    )
)
 
 chain = LLMChain(prompt=prompt_template, llm=llm)
 generated_text = chain.run({
    "negative": sentiment['negative'],
    "neutral": sentiment['neutral'],
    "positive": sentiment['positive'],
    "average": sentiment['average'],
    "decision": decision,
    "companyName": companyName,
    "history": history,
    "sentimenttext":sentiment_text
})
 return generated_text

User input

The user voice input should be like "Recommend me a stock on google" and it will work with the organizations set up in provided company-ticker.csv file.

Please reach out to me if any errors due to configuration of apikeys


In [54]:
import speech_recognition as sr
from playsound import playsound
from gtts import gTTS

def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        #st.write("Listening for your investment query...")
        print("Listening for your investment query...")
        audio = recognizer.listen(source)      
        try:
            text = recognizer.recognize_google(audio)
            print("You said:", text)
            #st.write("You said:", text)
            return text
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
            #st.write("Google Speech Recognition could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results; {e}")
            #st.write(f"Could not request results; {e}")
    return ""

speech_text = recognize_speech()
if speech_text:
        company_name = extract_companyName(speech_text)
        if company_name is not None:
            print(company_name)
            ticker = extract_tickerFromExcel(company_name)
            print(ticker)
            if ticker:
                real_time_data = fetch_realtime_data(ticker)
                if real_time_data.empty:
                    print(f"No real-time data found for ticker {ticker}.")

                current_price = real_time_data['Close'].values[-1]
                
                news_articles = fetch_news(extract_companyName(speech_text))
                if not news_articles:
                    print(f"No news articles found for {company_name}.")
                sentiment = analyze_sentiment(news_articles)
                decision = predict_stock_movement(sentiment)
                print(f"ticker: {ticker}")
                print(f"Current Price: {current_price}")
                symbol = ticker 
                stockHistory = fetch_daily_stock_data(symbol)
                if stockHistory is not None:
                    print("Daily Data:")
                    print(stockHistory.tail(10))  
                print(f"Sentiment Analysis: {sentiment}")
                print(f"Decision: {decision}")
                summary = generate_summary(sentiment, decision,extract_companyName(speech_text),stockHistory.tail(10))
                print("\nSummary:")
                if summary is not None:
                    for line in summary.split('.'):
                            if line.strip(): 
                                print(line.strip() + '.')
                    #text_to_speech(summary,f"summary_{company_name}.mp3")
                else:
                 summary = "Sorry could not fetch recommendation. please try again later" 
                 
            else:
              print("Could not extract stock ticker from your query.")
        else:
            print("Provide suitable inputs related to recommending a stock / organization data not available with us.")
else:
        print("No input detected.")


Listening for your investment query...
You said: recommend me a stock on Tesla
Tesla
TSLA


[*********************100%%**********************]  1 of 1 completed


apikey of news:8241d031a740409ebfbbddb45d0bcacb
https://newsapi.org/v2/everything?q=Tesla&domains=bloomberg.com,cnbc.com,financialpost.com,financialexpress.com,marketwatch.com,reuters.com,wsj.com,yahoo.com&sortBy=publishedAt&apiKey=8241d031a740409ebfbbddb45d0bcacb
